In [44]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [45]:
# Load data from a single file
def load_data(data_path):
    df = pd.read_csv(data_path)
    data = df['filename'].tolist()
    labels = df['label'].tolist()
    return data, labels

In [46]:
# Preprocess audio data
def preprocess_audio(file_path, target_duration=10):
    audio, _ = librosa.load(file_path, sr=None)

    target_length = int(target_duration * _)
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)))
    else:
        audio = audio[:target_length]

    mfccs = librosa.feature.mfcc(y=audio, sr=_, n_mfcc=13)
    return mfccs

In [47]:
# Model architecture
def build_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [48]:
# Train the model
def train_model(X_train, y_train, X_val, y_val):
    model = build_model(input_shape=X_train.shape[1:])
    optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Data Augmentation
    datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
    datagen.fit(X_train)

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Model Training
    model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_val, y_val), epochs=20, callbacks=[early_stopping], verbose=1)

    # Save the trained model to a file
    model.save('new_model.h5')

    return model


In [49]:
# Main function
def main():

    train_data_path = 'Hindi_train.csv'
    test_data_path = 'Hindi_test.csv'

    # Load training data
    data_train, labels_train = load_data(train_data_path)

    # Load testing data
    data_test, labels_test = load_data(test_data_path)

    # Convert string labels to integers
    y_train = np.array([1 if label == 'Yes' else 0 for label in labels_train])
    y_test = np.array([1 if label == 'Yes' else 0 for label in labels_test])

    # Combine datasets for k-fold cross-validation
    data_combined = data_train + data_test
    labels_combined = y_train.tolist() + y_test.tolist()

    # Convert to numpy arrays
    X_combined = np.array([preprocess_audio(os.path.join('./audios/', filename)) for filename in tqdm(data_combined, desc="Processing Audio")])
    y_combined = np.array(labels_combined)

    # Initialize cross-validator
    n_splits = 5  # You can adjust the number of folds
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold = 1
    accuracies = []
    all_true_labels = []
    all_pred_labels = []

    for train_index, val_index in skf.split(X_combined, y_combined):
        X_train, X_val = X_combined[train_index], X_combined[val_index]
        y_train, y_val = y_combined[train_index], y_combined[val_index]

        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]

        # Train the model
        model = train_model(X_train, y_train, X_val, y_val)

        # Predict on the validation set
        y_pred_probs = model.predict(X_val)
        y_pred = (y_pred_probs > 0.5).astype(int)

        # Evaluate accuracy
        accuracy = accuracy_score(y_val, y_pred)
        print(f"Fold {fold} - Validation Accuracy: {accuracy * 100:.2f}%")

        # Collect true and predicted labels for confusion matrix
        all_true_labels.extend(y_val)
        all_pred_labels.extend(y_pred.flatten())

        accuracies.append(accuracy)
        fold += 1

    # Calculate and print the overall accuracy
    overall_accuracy = np.mean(accuracies)
    print(f"\nOverall Cross-Validation Accuracy: {overall_accuracy * 100:.2f}%")

    # Calculate and print the confusion matrix
    cm = confusion_matrix(all_true_labels, all_pred_labels)
    print("\nConfusion Matrix:")
    print(cm)

    print("Cross-validation completed.")

In [50]:
if __name__ == "__main__":
    main()

Processing Audio:   0%|          | 4/1191 [00:00<00:36, 32.72it/s]

Processing Audio: 100%|██████████| 1191/1191 [01:34<00:00, 12.65it/s]


Epoch 1/20
30/30 [==============================] - 13s 270ms/step - loss: 1.4997 - accuracy: 0.6008 - val_loss: 4.0774 - val_accuracy: 0.5858
Epoch 2/20
30/30 [==============================] - 6s 193ms/step - loss: 1.5338 - accuracy: 0.5903 - val_loss: 2.6712 - val_accuracy: 0.6360
Epoch 3/20
30/30 [==============================] - 6s 183ms/step - loss: 1.3484 - accuracy: 0.6176 - val_loss: 3.1645 - val_accuracy: 0.5858
Epoch 4/20
30/30 [==============================] - 5s 168ms/step - loss: 1.2759 - accuracy: 0.6113 - val_loss: 1.7700 - val_accuracy: 0.6109
Epoch 5/20
30/30 [==============================] - 6s 182ms/step - loss: 1.2801 - accuracy: 0.6071 - val_loss: 1.2110 - val_accuracy: 0.6067
Epoch 6/20
30/30 [==============================] - 6s 205ms/step - loss: 1.0784 - accuracy: 0.6292 - val_loss: 0.7175 - val_accuracy: 0.7238
Epoch 7/20
30/30 [==============================] - 7s 219ms/step - loss: 1.0666 - accuracy: 0.6040 - val_loss: 1.3536 - val_accuracy: 0.6109
Epoch

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 45ms/step
Fold 1 - Validation Accuracy: 72.38%
Epoch 1/20
30/30 [==============================] - 11s 252ms/step - loss: 1.7889 - accuracy: 0.5677 - val_loss: 1.1526 - val_accuracy: 0.6597
Epoch 2/20
30/30 [==============================] - 7s 219ms/step - loss: 1.4243 - accuracy: 0.5719 - val_loss: 3.1129 - val_accuracy: 0.5462
Epoch 3/20
30/30 [==============================] - 7s 240ms/step - loss: 1.3866 - accuracy: 0.6139 - val_loss: 1.1294 - val_accuracy: 0.6723
Epoch 4/20
30/30 [==============================] - 8s 252ms/step - loss: 1.2812 - accuracy: 0.6306 - val_loss: 1.0631 - val_accuracy: 0.6807
Epoch 5/20
30/30 [==============================] - 7s 221ms/step - loss: 1.2496 - accuracy: 0.5897 - val_loss: 3.3268 - val_accuracy: 0.6008
Epoch 6/20
30/30 [==============================] - 7s 223ms/step - loss: 1.0920 - accuracy: 0.6118 - val_loss: 2.3204 - val_accuracy: 0.6134
Epoch 7/20
30/30 [==============================] - 7s 224

...

8/8 [==============================] - 1s 60ms/step
Fold 3 - Validation Accuracy: 63.87%
Epoch 1/20
30/30 [==============================] - 12s 265ms/step - loss: 1.6043 - accuracy: 0.5666 - val_loss: 8.3861 - val_accuracy: 0.4664
Epoch 2/20
30/30 [==============================] - 6s 205ms/step - loss: 1.5357 - accuracy: 0.6044 - val_loss: 4.1614 - val_accuracy: 0.5168
Epoch 3/20
30/30 [==============================] - 6s 190ms/step - loss: 1.3170 - accuracy: 0.6128 - val_loss: 2.0689 - val_accuracy: 0.6891
Epoch 4/20
30/30 [==============================] - 6s 181ms/step - loss: 1.2269 - accuracy: 0.5719 - val_loss: 1.2521 - val_accuracy: 0.7059
Epoch 5/20
30/30 [==============================] - 5s 177ms/step - loss: 1.0884 - accuracy: 0.6201 - val_loss: 1.4779 - val_accuracy: 0.6807
Epoch 6/20
30/30 [==============================] - 6s 181ms/step - loss: 1.1578 - accuracy: 0.5929 - val_loss: 1.2257 - val_accuracy: 0.6975
Epoch 7/20
30/30 [==============================] - 5s 181

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 0s 33ms/step
Fold 4 - Validation Accuracy: 71.01%
Epoch 1/20
30/30 [==============================] - 8s 203ms/step - loss: 1.2590 - accuracy: 0.6139 - val_loss: 2.5633 - val_accuracy: 0.7101
Epoch 2/20
30/30 [==============================] - 5s 180ms/step - loss: 1.3721 - accuracy: 0.6243 - val_loss: 2.1784 - val_accuracy: 0.6807
Epoch 3/20
30/30 [==============================] - 5s 164ms/step - loss: 1.5898 - accuracy: 0.5845 - val_loss: 1.9969 - val_accuracy: 0.6933
Epoch 4/20
30/30 [==============================] - 6s 187ms/step - loss: 1.3825 - accuracy: 0.6023 - val_loss: 1.1641 - val_accuracy: 0.7017
Epoch 5/20
30/30 [==============================] - 6s 187ms/step - loss: 1.1180 - accuracy: 0.6139 - val_loss: 1.9941 - val_accuracy: 0.5546
Epoch 6/20
30/30 [==============================] - 6s 196ms/step - loss: 1.0988 - accuracy: 0.6275 - val_loss: 1.5007 - val_accuracy: 0.6933
Epoch 7/20
30/30 [==============================] - 6s 188m

/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 58ms/step
Fold 5 - Validation Accuracy: 73.11%

Overall Cross-Validation Accuracy: 69.69%

Confusion Matrix:
[[424 211]
 [150 406]]
Cross-validation completed.
